In [8]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import cv2
import imageio
import os
from project.utils import labels2binary, kernel, seg_weights_2D, reindex_cell_labels, tracking_weights, rangescale
from scipy.ndimage import binary_fill_holes
import skimage.morphology as morph
from skimage.filters import sobel, gaussian
from skimage.measure import label

In [12]:
def custom_weights_1(seg, sigma_gaussian = 0.8, n_rounds_gaussian=50, quant_filt = 0.85, final_power = 6):
    '''
    Custom Weights Function
    ______________________________________________________________________________
    Function to generate weights that highlight boarders between neighboring cells 
    Based principally on Sobel filter, gaussian blur and filtering high values

    Parameters
    ----------
    seg : 2D array
        Tracking output mask.
    sigma_gaussian : float, optional
        Value controlling gaussian blur intensity
    n_rounds_gaussian : int, optional
        Number of times we iteratively apply gaussian blur
    quant_filt : float, optional
        Quantile used to filter high intensity value
    final_power : float, optional
        power to elevate final results to increase contrasts
    
    Returns
    -------
    weights : 2D array
        Tracking weights map.

    '''
    # First Sobel filtering to make cell boarders appear
    sob_s = sobel(seg)
    
    # First we apply multiple rounds of gaussian blur
    for i in range(n_rounds_gaussian):
        sob_s += gaussian(sob_s, sigma=sigma_gaussian, preserve_range=True)

    # Second, we only keep highest values (tend to overrepresent intersection between cells)
    quant_threshold = np.quantile(sob_s[sob_s > 0], q=quant_filt)
    sob_s[sob_s < quant_threshold] = 0
    # We give pixel inside cells a minimum value
    cell_score = min(sob_s[sob_s > 0])
    sob_s[seg == 1] += cell_score

    # We reverse the image and do similar filtering : This allow to improve signal on pixels between cells
    sob_s[sob_s > 0] = 1/sob_s[sob_s > 0]
    quant_threshold = np.quantile(sob_s[sob_s > 0], q=quant_filt)
    sob_s[sob_s > quant_threshold] = 0
    cell_score = min(sob_s[sob_s > 0])
    sob_s[seg == 1] += cell_score
    
    # We reverse again to the original 
    sob_s[sob_s > 0] = 1/sob_s[sob_s > 0]

    # We expend to a power to exagerate differences 
    sob_s = sob_s**final_power
    
    # Normalize between 0 and 255
    sob_s = (sob_s - np.min(sob_s))/(np.max(sob_s) - np.min(sob_s)) * 255

    return(sob_s)

#### BUILD TRAINING SET FOR CELL TRACKING (FROM MANUAL ANNOTATIONS)

Hereafter, we load our manually curated annotation of cell tracking and we build Delta-compatible training set

In [21]:
# File and folders used 
cell_img_file = "data/_fullmovie_images.tif"
cell_seg_file = "data/_fullmovie_segmentation.tif"
annot_file = "data/_cell_tracking_manual_annotations.tif"

First we load raw images which are also used by Delta model : 

In [23]:
# First, we load the raw images :
_, cell_img = cv2.imreadmulti(cell_img_file, [], cv2.IMREAD_ANYDEPTH)
# Create numpy to store final values
img = np.zeros((125, cell_img[0].shape[0], cell_img[0].shape[1])).astype('float32')
# Get min and max from whole movie to normalize image intensity values
max_full_img = np.max(cell_img) ; min_full_img = np.min(cell_img)
# iterate over frames
for i in range(len(cell_img)):
    I = cell_img[i].copy().astype('float32')
    I_norm = (I - min_full_img)/(max_full_img - min_full_img) # normalization accross all frames and all images
    img[i] = (I_norm.copy()*255).astype('uint8') # to subset : training [0:512, 0:512], testing [1024:1536, 0:512] 

We also load the full segmentation

In [22]:
# First, we load the raw images :
_, cell_seg_l = cv2.imreadmulti(cell_seg_file, [], cv2.IMREAD_ANYDEPTH)
cell_mask = np.zeros((125, cell_seg_l[0].shape[0], cell_seg_l[0].shape[1]))
for f in range(len(cell_seg_l)):
    labs = cell_seg_l[f].copy()
    mask = labels2binary(labs)
    mask = binary_fill_holes(mask).astype(np.uint8)
    mask[mask > 0] = 1
    cell_mask[f] = mask

Then we can load our manually annotated cell segmentation and use them as label and as binary mask

In [24]:
# Load curated annotations
_, annot_l = cv2.imreadmulti(annot_file, [], cv2.IMREAD_ANYDEPTH)
# From manual annotation, get labels and binary mask
annot_reidx = np.zeros((125,annot_l[0].shape[0],annot_l[0].shape[1]))
for i in range(len(annot_l)):
    this_lab = reindex_cell_labels(annot_l[i].copy()).astype('uint16') # we keep labs 
    # Record annotation movie (contains only annotated tracked cells)
    annot_reidx[i] = this_lab.copy()
    annot_reidx[i][annot_reidx[i] <= 1] = 0

Hereafter is the loop that will iterate over curated annotations, to choose pair of frames with identical cell annotations. These annotations were done in a way so that unique labels always refer to single cells accross frames. This only special cases are cell division, where there is one cell at frame t connected with two cells at frame t+1. 

Once the pair of cells has been detected, all the files necessary for cell tracking are written in their respective folder, as Delta is requiring them to be :
- previmg : image at time t-1 (input)
- img : image at time t (input)
- seg : segmentation binary mask showing only cell of interest at time t-1 (input)
- segall : segmentation binary mask showing all cells in images at time t (input)
- wei : weights of cells at time time t (weights the loss)
- mot_dau : segmentation binary mask showing only cell of interest at time t (output of model)

All images have to be centered on a cell of interest with size (256,256) around it. 6 images are therefore produced for each pair of cell accross two time points. The curated training set encompass a total of 1454 x 6 images. 

__Warning : Next cell might take very long to compute__

_to reduce the time it's possible to build a tiny training set by changing the range over the frame range(124), into something like range(100,110). But as we believe our training set is already too small to produce good results, this is maybe not a good idea_

In [29]:
out_dir = "data/training_tracking_test/"
if not os.path.exists(out_dir):
    os.mkdir(out_dir) ; os.mkdir(out_dir + "img") ; os.mkdir(out_dir + "previmg") ; os.mkdir(out_dir + "seg")
    os.mkdir(out_dir + "segall") ; os.mkdir(out_dir + "wei") ; os.mkdir(out_dir + "mot_dau")
    
idx=1
# Iterate over frames : 
for f in range(124):
    print(f)
    
    # Iterate over unique cell labels found in frame f in our manually annotated data
    uniq_labs = np.unique(annot_reidx[f])
    for lab in uniq_labs[uniq_labs != 0]:
        if any(np.unique(annot_reidx[f+1]) == lab):
            print('frame {0}, label {1}'.format(f, lab))
            
            # Load cell track manual annotations for frame (t)
            anno2work = annot_reidx[f].copy() 
            # Set current cell label to 1 and every other cell to 0
            anno2work[anno2work != lab] = 0 ; anno2work[anno2work == lab] = 1 
            
            # Load annotation for next time frame (t+1)
            anno2work_next = annot_reidx[f+1].copy()
            # Set current cell label to 1 and every other cell to 0
            anno2work_next[anno2work_next != lab] = 0 ; anno2work_next[anno2work_next == lab] = 1
                
            # Load the segmentation binary mask of all cells at time t+1 
            all_seg_fp1 = cell_mask[f+1].copy()
            prev_img = img[f].copy()
            curr_img = img[f+1].copy()

            ### BUILD IMAGE CENTERED ON CELL OF INTEREST ###
            # Get mean pixel position on x and y axis
            x_mean = int(np.where(anno2work == 1)[0].mean())
            y_mean = int(np.where(anno2work == 1)[1].mean())
            # Get coordinates on the main segmentation binary mask
            x_start = int(max(x_mean - (256/2), 0))
            x_end = int(min(x_mean + (256/2), img[0].shape[0]))
            y_start = int(max(y_mean - (256/2), 0))
            y_end = int(min(y_mean + (256/2), img[0].shape[1]))
            # Hereafter the coordinates we calculate for the (256,256) subset image
            x_reform_start = int(abs(min(x_mean - (256/2), 0)))
            y_reform_start = int(abs(min(y_mean - (256/2), 0)))
            x_shift, y_shift = 0, 0
            if x_mean + (256/2) > img[0].shape[0]: 
                x_shift = int(img[0].shape[0] - (x_mean + (256/2)))
            if y_mean + (256/2) > img[0].shape[1]: 
                y_shift = int(img[0].shape[1] - (y_mean + (256/2)))
            x_reform_end = int(abs(min(x_mean + (256/2), 256))) + x_shift + x_reform_start
            y_reform_end = int(abs(min(y_mean + (256/2), 256))) + y_shift + y_reform_start

            # This is the (256,256) segmentation image that matches the prediction (256,256) image
            # Thanks to all coordinates found above we can connect our two images with cell of interest
            centered_seg = np.zeros((256,256)) ; centered_seg_next = np.zeros((256,256))
            centered_all_seg = np.zeros((256,256)) ;  centered_img = np.zeros((256,256))
            centered_prev_img = np.zeros((256,256)) ;
            centered_seg[x_reform_start:x_reform_end, y_reform_start:y_reform_end] = anno2work[x_start:x_end,y_start:y_end].copy()
            centered_seg_next[x_reform_start:x_reform_end, y_reform_start:y_reform_end] = anno2work_next[x_start:x_end,y_start:y_end].copy()
            centered_all_seg[x_reform_start:x_reform_end, y_reform_start:y_reform_end] = all_seg_fp1[x_start:x_end,y_start:y_end].copy()
            centered_prev_img[x_reform_start:x_reform_end, y_reform_start:y_reform_end] = prev_img[x_start:x_end,y_start:y_end].copy()
            centered_img[x_reform_start:x_reform_end, y_reform_start:y_reform_end] = curr_img[x_start:x_end,y_start:y_end].copy()

            # Make sure not to consider cases where 2 cells become 1 cell
            # Make sure not to consider 1 cell becoming 0 cell
            # One condition to check : if n_cell at t is greater that n_cell at t+1, skip
            n_cell_t = len(np.unique(label(centered_seg)))-1
            n_cell_tp1 = len(np.unique(label(centered_seg_next)))-1
            if n_cell_t > n_cell_tp1:
                continue
                
            # Build weight with custom weight function
            wei = custom_weights_1(centered_seg_next.copy())
            wei[centered_seg_next == 1] = 100 # add minimal weight to cell of interest
            wei[centered_seg_next == 0] = 1
            wei_01 = ( wei - np.min(wei) ) / (np.max(wei) - np.min(wei))
            wei_01 *= 255
            wei_int = np.round(wei_01).astype('uint8')
            
            # Get output name
            out_name = "Sample0000" + str(idx) + ".png"
            if idx > 9:
                out_name = "Sample000" + str(idx) + ".png"
            if idx > 99:
                out_name = "Sample00" + str(idx) + ".png"
            if idx > 999:
                out_name = "Sample0" + str(idx) + ".png"
            if idx > 9999:
                out_name = "Sample" + str(idx) + ".png"
                                  
            # Write results in folder 
            cv2.imwrite(out_dir + "img/" + out_name, centered_img)
            cv2.imwrite(out_dir + "previmg/" + out_name, centered_prev_img)
            cv2.imwrite(out_dir + "mot_dau/" + out_name, centered_seg_next)
            cv2.imwrite(out_dir + "seg/" + out_name, centered_seg)
            cv2.imwrite(out_dir + "segall/" + out_name, centered_all_seg)
            cv2.imwrite(out_dir + "wei/" + out_name, wei_int)
            
            idx+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
frame 32, label 2.0
33
frame 33, label 2.0
34
frame 34, label 2.0
35
frame 35, label 2.0
36
frame 36, label 2.0
37
frame 37, label 2.0
frame 37, label 3.0
38
frame 38, label 2.0
frame 38, label 3.0
39
frame 39, label 2.0
frame 39, label 3.0
40
frame 40, label 2.0
frame 40, label 3.0
41
frame 41, label 2.0
frame 41, label 3.0
frame 41, label 4.0
42
frame 42, label 2.0
frame 42, label 3.0
frame 42, label 4.0
43
frame 43, label 2.0
frame 43, label 3.0
frame 43, label 4.0
44
frame 44, label 2.0
frame 44, label 3.0
frame 44, label 4.0
45
frame 45, label 2.0
46
frame 46, label 2.0
47
frame 47, label 2.0
frame 47, label 3.0
48
frame 48, label 2.0
frame 48, label 3.0
49
frame 49, label 2.0
frame 49, label 3.0
frame 49, label 4.0
50
frame 50, label 2.0
frame 50, label 3.0
frame 50, label 4.0
51
frame 51, label 2.0
frame 51, label 3.0
frame 51, label 4.0
52
frame 52, label 2.0
frame 52, label 3.0
frame 52, l

frame 84, label 20.0
frame 84, label 21.0
frame 84, label 22.0
frame 84, label 23.0
frame 84, label 24.0
frame 84, label 25.0
85
frame 85, label 2.0
frame 85, label 3.0
frame 85, label 4.0
frame 85, label 5.0
frame 85, label 6.0
frame 85, label 7.0
frame 85, label 8.0
frame 85, label 9.0
frame 85, label 10.0
frame 85, label 11.0
frame 85, label 12.0
frame 85, label 13.0
frame 85, label 14.0
frame 85, label 15.0
frame 85, label 16.0
frame 85, label 17.0
frame 85, label 18.0
frame 85, label 19.0
frame 85, label 20.0
frame 85, label 21.0
frame 85, label 22.0
frame 85, label 23.0
frame 85, label 24.0
frame 85, label 25.0
frame 85, label 26.0
86
frame 86, label 2.0
frame 86, label 3.0
frame 86, label 4.0
frame 86, label 5.0
frame 86, label 6.0
frame 86, label 7.0
frame 86, label 8.0
frame 86, label 9.0
frame 86, label 10.0
frame 86, label 11.0
frame 86, label 12.0
frame 86, label 13.0
frame 86, label 14.0
frame 86, label 15.0
frame 86, label 16.0
frame 86, label 17.0
frame 86, label 18.0
fr

frame 95, label 27.0
frame 95, label 28.0
frame 95, label 29.0
frame 95, label 30.0
frame 95, label 31.0
frame 95, label 32.0
frame 95, label 33.0
frame 95, label 34.0
frame 95, label 35.0
frame 95, label 36.0
frame 95, label 37.0
frame 95, label 38.0
frame 95, label 39.0
frame 95, label 40.0
frame 95, label 41.0
frame 95, label 42.0
frame 95, label 43.0
frame 95, label 44.0
frame 95, label 45.0
frame 95, label 46.0
frame 95, label 47.0
96
frame 96, label 2.0
frame 96, label 3.0
frame 96, label 4.0
frame 96, label 5.0
frame 96, label 6.0
frame 96, label 7.0
frame 96, label 8.0
frame 96, label 9.0
frame 96, label 10.0
frame 96, label 11.0
frame 96, label 12.0
frame 96, label 13.0
frame 96, label 14.0
frame 96, label 15.0
frame 96, label 16.0
frame 96, label 17.0
frame 96, label 18.0
frame 96, label 19.0
frame 96, label 20.0
frame 96, label 21.0
frame 96, label 22.0
frame 96, label 23.0
frame 96, label 24.0
frame 96, label 25.0
frame 96, label 26.0
frame 96, label 27.0
frame 96, label 28

frame 104, label 19.0
frame 104, label 20.0
frame 104, label 21.0
frame 104, label 22.0
frame 104, label 23.0
frame 104, label 24.0
frame 104, label 25.0
frame 104, label 26.0
frame 104, label 27.0
frame 104, label 28.0
frame 104, label 29.0
frame 104, label 30.0
frame 104, label 31.0
frame 104, label 32.0
frame 104, label 33.0
frame 104, label 34.0
frame 104, label 35.0
frame 104, label 36.0
frame 104, label 37.0
frame 104, label 38.0
frame 104, label 39.0
frame 104, label 40.0
frame 104, label 41.0
frame 104, label 42.0
frame 104, label 43.0
frame 104, label 44.0
frame 104, label 45.0
frame 104, label 46.0
frame 104, label 47.0
105
frame 105, label 2.0
frame 105, label 3.0
frame 105, label 4.0
frame 105, label 5.0
frame 105, label 6.0
frame 105, label 7.0
frame 105, label 8.0
frame 105, label 9.0
frame 105, label 10.0
frame 105, label 11.0
frame 105, label 12.0
frame 105, label 13.0
frame 105, label 14.0
frame 105, label 15.0
frame 105, label 16.0
frame 105, label 17.0
frame 105, lab

frame 112, label 18.0
frame 112, label 19.0
frame 112, label 20.0
frame 112, label 21.0
frame 112, label 22.0
frame 112, label 23.0
frame 112, label 24.0
frame 112, label 25.0
frame 112, label 26.0
frame 112, label 27.0
frame 112, label 28.0
frame 112, label 29.0
frame 112, label 30.0
frame 112, label 31.0
frame 112, label 32.0
frame 112, label 33.0
frame 112, label 34.0
frame 112, label 35.0
frame 112, label 36.0
frame 112, label 37.0
frame 112, label 38.0
frame 112, label 39.0
frame 112, label 40.0
frame 112, label 41.0
frame 112, label 42.0
frame 112, label 43.0
frame 112, label 44.0
frame 112, label 45.0
frame 112, label 46.0
frame 112, label 47.0
frame 112, label 48.0
frame 112, label 49.0
frame 112, label 50.0
frame 112, label 51.0
frame 112, label 52.0
frame 112, label 53.0
frame 112, label 54.0
frame 112, label 55.0
frame 112, label 56.0
frame 112, label 57.0
frame 112, label 58.0
frame 112, label 59.0
frame 112, label 60.0
frame 112, label 61.0
113
frame 113, label 2.0
frame 1

frame 117, label 48.0
frame 117, label 49.0
frame 117, label 50.0
frame 117, label 51.0
frame 117, label 52.0
frame 117, label 53.0
frame 117, label 54.0
frame 117, label 55.0
frame 117, label 56.0
frame 117, label 57.0
frame 117, label 58.0
frame 117, label 59.0
frame 117, label 60.0
frame 117, label 61.0
frame 117, label 62.0
frame 117, label 63.0
frame 117, label 64.0
frame 117, label 65.0
frame 117, label 66.0
frame 117, label 67.0
frame 117, label 68.0
frame 117, label 69.0
frame 117, label 70.0
frame 117, label 71.0
frame 117, label 72.0
frame 117, label 73.0
frame 117, label 74.0
frame 117, label 75.0
frame 117, label 76.0
frame 117, label 77.0
frame 117, label 78.0
frame 117, label 79.0
118
frame 118, label 2.0
frame 118, label 3.0
frame 118, label 4.0
frame 118, label 5.0
frame 118, label 6.0
frame 118, label 7.0
frame 118, label 8.0
frame 118, label 9.0
frame 118, label 10.0
frame 118, label 11.0
frame 118, label 12.0
frame 118, label 13.0
frame 118, label 14.0
frame 118, lab

frame 122, label 36.0
frame 122, label 37.0
frame 122, label 38.0
frame 122, label 39.0
frame 122, label 40.0
frame 122, label 41.0
frame 122, label 42.0
frame 122, label 43.0
frame 122, label 44.0
frame 122, label 45.0
frame 122, label 46.0
frame 122, label 47.0
frame 122, label 48.0
frame 122, label 49.0
frame 122, label 50.0
frame 122, label 51.0
frame 122, label 52.0
frame 122, label 53.0
frame 122, label 54.0
frame 122, label 55.0
frame 122, label 56.0
frame 122, label 57.0
frame 122, label 58.0
frame 122, label 59.0
frame 122, label 60.0
frame 122, label 61.0
frame 122, label 62.0
frame 122, label 63.0
frame 122, label 64.0
frame 122, label 65.0
frame 122, label 66.0
frame 122, label 67.0
frame 122, label 68.0
frame 122, label 69.0
frame 122, label 70.0
frame 122, label 71.0
frame 122, label 72.0
frame 122, label 73.0
frame 122, label 74.0
frame 122, label 75.0
frame 122, label 76.0
123
frame 123, label 2.0
frame 123, label 3.0
frame 123, label 4.0
frame 123, label 5.0
frame 123,

#### Prepare input data for predictions 

Write the image / segmentation in the same way they were built for building training set (same normalization on image, same segmentation, etc..). Delta needs output in format type `Position01Channe01Frame00001` for tracking predictions.

In [30]:
outdir = 'data/input_for_predictions/tracking/'
if not os.path.exists(outdir):
    os.mkdir(outdir) ; 
if not os.path.exists(outdir + "img"):
    os.mkdir(outdir + "img") ; os.mkdir(outdir + "seg")

In [31]:
idx = 0
compilation = np.zeros((125,300,300)) # here we generate a "compilation" file to give as input
for f in range(len(img)):
    # Get output name
    out_name = "Position01Channe01Frame0000" + str(idx) + ".png"
    if idx > 9:
        out_name = "Position01Channe01Frame000" + str(idx) + ".png"
    if idx > 99:
        out_name = "Position01Channe01Frame00" + str(idx) + ".png"
        
    # Write image in folder : 
    cv2.imwrite(outdir+"img/"+out_name, img[f][0:300, 0:300].copy())
    
    # Prepare segmentation : 
    seg_c = cell_mask[f][0:300, 0:300].copy()
    mask = labels2binary(seg_c)
    mask = binary_fill_holes(mask).astype(np.uint8)
    mask[mask > 0] = 1
    compilation[f] = mask
    # Write segmentation : 
    cv2.imwrite(outdir+"seg/"+out_name, mask)
    idx+=1